# Recurrent repeat expansions in human cancer genomes

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import seaborn as sns

from scipy import stats

import statsmodels.api as sm
import statsmodels

from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts

from pyliftover import LiftOver

from pathlib import Path

%matplotlib inline
sns.set_context('paper', font_scale=1.1)
sns.set_style('ticks')

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Data preparation & helpers

In [ ]:
ROOT = Path('/labs/mpsnyder/gerwin/TR_analysis/')
CANCER_EXP_PATH = ROOT/'ashwini/V1.0/expt/cancer_v100/'
CANCER_LIST_PATH =ROOT/'ashwini/V1.0/src/cancerlist.v100'

In [ ]:
CANCER_NAMES = [name.strip() for name in open("data/input/cancerlist.txt").readlines()]
CHROMS = [f"chr{i}" for i in range(1, 22+1)] + ['chrX', 'chrY']
RRE = pd.read_table("data/input/rre.tsv")
RRE["chr"] = "chr" + RRE["chr"]

In [ ]:
REF_GENOME = "" # LEFT HERE
BAM_CAKI_1 = "/home/rashid/TR_analysis/rashid/dna_meth_rre/pbmm2/Caki-1.5mc.aligned.GRCh37.bam"
BAM_786_O = "/home/rashid/TR_analysis/rashid/dna_meth_rre/pbmm2/786-O.5mc.aligned.GRCh37.bam"

In [ ]:
def calc_effect_size(control, experiment):
    # https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/cohens-d/
    pooled_sd = ((experiment.std() ** 2 + control.std() ** 2) / 2) ** 0.5
    mean_diff =  experiment.mean() - control.mean()
    return mean_diff / pooled_sd

## Main Figures

### (2E) Distance of rREs to the nearest cCRE

#### Download catalogs and sort

The simple repeats catalogue was downloaded from [the UCSC genome broswer](https://genome.ucsc.edu/cgi-bin/hgTables). The cCRE catalog (Registry V3 in hg38 coordinates) was downloaded from the [ENCODE project](https://screen.encodeproject.org/). Then, it was lifted over to hg19 using the online [liftover tool](https://genome.ucsc.edu/cgi-bin/hgLiftOver).

#### Clean and adjust size of simple repeats

In [ ]:
simplerepeats = pd.read_table("data/input/simplerepeats.bed", header=None, usecols=range(3))
simplerepeats.columns = ["chr", "start", "stop"]

# only include repeats within chr1 to chrY
simplerepeats = simplerepeats.loc[simplerepeats["chr"].isin(CHROMS)]

Save as BED file and sort.

In [ ]:
simplerepeats.to_csv("data/input/simplerepeats_clean.bed", index=False, header=None, sep="\t")
!sort -k1,1 -k2,2n data/input/simplerepeats_clean.bed > data/input/simplerepeats_clean.sorted.bed

#### Find distance of closest cCRE to rREs and Simple Repeats

In [ ]:
!module load bedtools && bedtools closest -d -t first -a "data/input/rre.bed" -b "data/input/ccre.sorted.bed" > "data/output/closest_rre_ccre.bed"

In [ ]:
!module load bedtools && bedtools closest -d -t first -a "data/input/simplerepeats_clean.sorted.bed" -b "data/input/ccre.sorted.bed" > "data/output/closest_simplerepeats_ccre.bed"

#### Prepare data

In [ ]:
rre_distance = pd.read_table("data/output/closest_rre_ccre.bed", header=None)
simple_distance = pd.read_table("data/output/closest_simplerepeats_ccre.bed", header=None)

# Extract distance column from bedtools output
distance_col = 9
rre_distance = rre_distance[[distance_col]].rename({distance_col: "distance"}, axis="columns")
simple_distance = simple_distance[[distance_col]].rename({distance_col: "distance"}, axis="columns")

rre_distance["type"] = "rREs"
simple_distance["type"] = "Simple Repeats"

# Convert distance to kbp
rre_distance["distance"] = rre_distance["distance"] / 10**3
simple_distance["distance"] = simple_distance["distance"] / 10**3

simple_distance = simple_distance[simple_distance["distance"] < 25000]

# Merge the two dataframes
data = pd.concat([rre_distance, simple_distance], ignore_index=True)

#### How many rREs directly overlap with a cCRE?

In [ ]:
len(rre_distance[rre_distance["distance"] == 0])

#### Calculate p-value

In [ ]:
_, p_value = stats.ttest_ind(rre_distance["distance"], simple_distance["distance"], equal_var=False)
print(f"p={p_value}")

#### Calculate effect size (Cohens's d)

In [ ]:
cohen_d = calc_effect_size(simple_distance["distance"], rre_distance["distance"])
print(f"d={cohen_d}")

#### Plot

In [ ]:
pdf = PdfPages("data/plot/2022-10-17-distance-to-nearest-ccre.pdf")

fig, ax = plt.subplots()
sns.boxplot(x="type", y="distance", order=["Simple Repeats", "rREs"], showfliers=False, width=0.25, data=data, ax=ax)
ax.set_title(f"Distance to Nearest cCRE\np={p_value}\nd={cohen_d}")
ax.set_xlabel("")
ax.set_ylabel("Distance (kbp)")

# ax.axhline(y=rre_distance["distance"].mean(), linestyle="-")
# ax.axhline(y=simple_distance["distance"].mean(), linestyle="--")

sns.despine()
plt.setp(ax.artists, edgecolor = 'k', facecolor='w')
plt.setp(ax.lines, color='k')
ax.grid(False)
plt.tight_layout()

pdf.savefig(transparent=True)
pdf.close()

### (3C) Distance to prostate cancer risk loci

The Prost-AdenoCA risk loci were obtained from [Schumacker et al.](https://www.nature.com/articles/s41588-018-0142-8). We extract the first three columns (chr, start, stop) from the file and sort it. Then, we extract the chromosomes the the risk loci are located in.

In [ ]:
!cat data/input/Prostate-AdenoCA-risk-loci-Schumaker-et-al.bed | awk '{print $1 "\t" $2 "\t" $3}' | sort -k1,1 -k2,2n > data/input/prost_risk_loci_clean.bed

#### Extract prostate and non-prostate rREs into BED files

In [ ]:
prost_rre = RRE.loc[RRE["cancer"] == "Prost-AdenoCA"].copy()
non_prost_rre = RRE.loc[~(RRE["cancer"] == "Prost-AdenoCA")].copy()

len(prost_rre), len(non_prost_rre)

In [ ]:
prost_rre[["chr", "start", "stop"]].to_csv("data/input/prost_rre.bed", index=False, header=None, sep="\t")
!sort -k1,1 -k2,2n data/input/prost_rre.bed > data/input/prost_rre.sorted.bed

In [ ]:
non_prost_rre[["chr", "start", "stop"]].to_csv("data/input/non_prost_rre.bed", index=False, header=None, sep="\t")
!sort -k1,1 -k2,2n data/input/non_prost_rre.bed > data/input/non_prost_rre.sorted.bed

#### Find distance

In [ ]:
!module load bedtools && bedtools closest -d -a "data/input/prost_rre.sorted.bed" -b "data/input/prost_risk_loci_clean.bed" > "data/output/closest_prost_rre_risk_loci.bed"

In [ ]:
!module load bedtools && bedtools closest -d -a "data/input/non_prost_rre.sorted.bed" -b "data/input/prost_risk_loci_clean.bed" > "data/output/closest_non_prost_rre_risk_loci.bed"

In [ ]:
!module load bedtools && bedtools closest -d -a "data/input/simplerepeats_clean.sorted.bed" -b "data/input/prost_risk_loci_clean.bed" > "data/output/closest_simplerepeats_prost_risk_loci.bed"

#### Prepare data

In [ ]:
prost_rre_distance = pd.read_table("data/output/closest_prost_rre_risk_loci.bed", header=None)
non_prost_rre_distance = pd.read_table("data/output/closest_non_prost_rre_risk_loci.bed", header=None)
simplerepeats_distance = pd.read_table("data/output/closest_simplerepeats_prost_risk_loci.bed", header=None)

In [ ]:
# Extract distance column from bedtools output
distance_col = 6
prost_rre_distance = prost_rre_distance[[distance_col]].rename({distance_col: "distance"}, axis="columns")
non_prost_rre_distance = non_prost_rre_distance[[distance_col]].rename({distance_col: "distance"}, axis="columns")
simplerepeats_distance = simplerepeats_distance[[distance_col]].rename({distance_col: "distance"}, axis="columns")

prost_rre_distance["type"] = "Prost-AdenoCA rREs"
non_prost_rre_distance["type"] = "Non-Prost-AdenoCA rREs"
simplerepeats_distance["type"] = "Simple Repeats"

# Convert distance to kbp
prost_rre_distance["distance"] = prost_rre_distance["distance"] / 10**3
non_prost_rre_distance["distance"] = non_prost_rre_distance["distance"] / 10**3
simplerepeats_distance["distance"] = simplerepeats_distance["distance"] / 10**3

# Loci that exist on chroms that don't have a risk loci get assigned the max distance
prost_rre_distance.loc[prost_rre_distance["distance"] < 0, "distance"] = prost_rre_distance["distance"].max()
non_prost_rre_distance.loc[non_prost_rre_distance["distance"] < 0, "distance"] = non_prost_rre_distance["distance"].max()
simplerepeats_distance.loc[simplerepeats_distance["distance"] < 0, "distance"] = simplerepeats_distance["distance"].max()

# Merge the two dataframes
data = pd.concat([prost_rre_distance, non_prost_rre_distance, simplerepeats_distance], ignore_index=True)

len(prost_rre_distance), len(non_prost_rre_distance), len(simplerepeats_distance)

#### Find p-value and effect size

In [ ]:
_, p_value_prost_rre = stats.ttest_ind(prost_rre_distance["distance"], simplerepeats_distance["distance"], equal_var=False)
_, p_value_non_prost_rre = stats.ttest_ind(non_prost_rre_distance["distance"], simplerepeats_distance["distance"], equal_var=False)
print(f"Prost-AdenoCA rREs p={p_value_prost_rre}")
print(f"Non-Prost-AdenoCA rREs p={p_value_non_prost_rre}")

In [ ]:
_, q_values, _, _ = statsmodels.stats.multitest.multipletests([p_value_prost_rre, p_value_non_prost_rre], method='fdr_bh')
q_value_prost_rre, q_value_non_prost_rre = q_values[0], q_values[1]
print(f"Prost-AdenoCA rREs q={q_value_prost_rre}")
print(f"Non-Prost-AdenoCA rREs q={q_value_non_prost_rre}")

In [ ]:
cohen_d_prost_rre = calc_effect_size(simplerepeats_distance["distance"], prost_rre_distance["distance"])
cohen_d_non_prost_rre = calc_effect_size(simplerepeats_distance["distance"], non_prost_rre_distance["distance"])

print(f"Prost-AdenoCA rREs d={cohen_d_prost_rre}")
print(f"Non-Prost-AdenoCA rREs d={cohen_d_non_prost_rre}")

#### Plot

In [ ]:
pdf = PdfPages("data/plot/2022-10-17-distance-to-prost-risk-loci.pdf")

fig, ax = plt.subplots()
sns.boxplot(x="type", y="distance", order=["Simple Repeats", "Non-Prost-AdenoCA rREs", "Prost-AdenoCA rREs"], showfliers=False, width=0.25, data=data, ax=ax)
ax.set_title(f"Distance to Nearest Prostate Risk Locus\nNon-Prost-AdenoCA p={p_value_non_prost_rre}, d={cohen_d_non_prost_rre}\nProst-AdenoCA p={p_value_prost_rre}, d={cohen_d_prost_rre}\nq={q_value_prost_rre} (for both)\n")
ax.set_xlabel("")
ax.set_ylabel("Distance (kbp)")

sns.despine()
plt.setp(ax.artists, edgecolor = 'k', facecolor='w')
plt.setp(ax.lines, color='k')
ax.grid(False)
plt.tight_layout()

pdf.savefig(transparent=True)
pdf.close()

### (4B) Long-read DNA sequencing visualization

#### Create BED file for TRGT

In [ ]:
df = pd.read_json("data/input/2022-Zhu-148-rRE-Annotations.json")

df[['chr', 'pos']] = df['ReferenceRegion'].str.split(":", expand=True)

df[['start', 'stop']] = df['pos'].str.split("-", expand=True)

df['struc'] = df['LocusStructure'].replace("\*", "n", regex=True) 

df['info'] = "ID=" + df["LocusId"] + ",STRUC=" + df['struc'] 

df['chr'] = df['chr'].replace("chr", "", regex=True)

trgt_bed = df[['chr', 'start', 'stop', 'info']]
trgt_bed.to_csv("data/input/rres_trtg.bed", sep="\t", header=None, index=False)

#### Download binaries

In [ ]:
!wget -O util/trvz.gz https://github.com/PacificBiosciences/trgt/releases/download/v0.3.2/trvz-v0.3.2-linux_x86_64.gz

In [ ]:
!wget -O util/trgt.gz https://github.com/PacificBiosciences/trgt/releases/download/v0.3.2/trgt-v0.3.2-linux_x86_64.gz

In [ ]:
!gunzip util/trvz.gz && gunzip util/trgt.gz

In [ ]:
!chmod +x util/trgt util/trvz

#### Run TRGT

To run TRGT, download the Caki-1 and 786-O short-read sequencing data (links found in paper), then use the following command to run the tool. Refer to the [TRGT documentation](https://github.com/PacificBiosciences/trgt/blob/main/docs/tutorial.md) for more details on running the tool.

```
./trgt --genome <REFRENCE_HG19> --repeats data/input/rres_trtg.bed --reads <BAM_FILE> --output-prefix <PREFIX>
```

#### Run TRVZ

```
./trvz --genome example/reference.fasta \
       --repeats example/repeat.bed \
       --vcf sample.sorted.vcf.gz \
       --spanning-reads sample.spanning.sorted.bam \
       --repeat-id TR1 \
       --image TR1.svg
```

### (4D) UGT2B7 Transcript Isoform Differential Expression

#### Download transcript counts from PCAWG

In [ ]:
!wget -O data/input/pcawg.rnaseq.transcript.expr.counts.tsv.gz https://dcc.icgc.org/api/v1/download?fn=/PCAWG/transcriptome/transcript_expression/pcawg.rnaseq.transcript.expr.tpm.tsv.gz

In [ ]:
!gunzip data/input/pcawg.rnaseq.transcript.expr.counts.tsv.gz

#### Prepare metadata for DESeq

In [ ]:
transcripts = ['ENST00000305231.7', 'ENST00000508661.1', 'ENST00000502942.1']

patient_file_path = CANCER_EXP_PATH/'{}/output/patientdata/{}_paired_{}_{}_{}_{}.txt'.format('Kidney-RCC', 'Kidney-RCC', 'AAAG', 4, 69929297, 69930148)
patients = pd.read_csv(patient_file_path, sep='\t')
patients = patients.rename(columns={'case_id': 'icgc_donor_id'})

metadata = pd.read_csv('data/input/rnaseq.extended.metadata.aliquot_id.V4.tsv', sep='\t')
metadata = metadata.rename(columns={'tumor.normal': 'sample_type'})    
metadata = metadata[metadata['sample_type'] == 'tumor']
metadata = metadata.merge(patients, on='icgc_donor_id')
metadata = metadata.rename({'aliquot_id': 'id'}, axis='columns')
metadata = metadata[["id", "exp"]]

metadata.to_csv("data/input/ugt2b7_expr_metadata.csv", index=False)

len(metadata)

#### Run DESeq

Run the R notebook `utils/Expression Analysis.ipynb` to complete the analysis.

## Extended Data Figures

### (2D) Estimated Frequency of rREs in the population

Here, I am first making a table of the average global-normalized anchored IRR value for each rRE. Then, I am joining that table with the table from Ryan Yuen which has the anchored IRR value for each rRE for each sample in the 1000 Genomes dataste plus others.

The bigger picture here is to find loci that are generally shorter than the read length in the population by looking at the average global-normalized anchored IRR value, and then calling samples in the 1000Genomes dataset with an IRR count higher than a threshold expanded (meaning that they are longer than the read length in that sample).

#### Settings

In [ ]:
PERCENTILE = 99

#### Make a table of avg Anchored IRR value for PCAWG

In [ ]:
tropic = pd.read_table('data/input/tropic_rre.tsv')
lrdn = pd.read_table('data/input/rre.tsv')
len(tropic), len(lrdn)

In [ ]:
merged = lrdn.merge(tropic, on=['chr', 'start', 'stop', 'motif', 'cancer'], suffixes=('_lrdn', '_tropic'))
merged

#### Calculate the average IRR value for control samples for each locus

In [ ]:
def get_control_irr_val_cutoff(repeat):
    # Initialize all IRR values of files to zero (only for control samples)
    cancer = repeat['cancer']
    manifest_path = f'data/input/EHdn_v097_Manifests/{cancer}.manifest.tsv'
    manifest = pd.read_csv(manifest_path, header=None, sep=' ')
    file_names = list(manifest[0])
    irr_values = {f_name: 0 for f_name in file_names if f_name.startswith('control_')}

    # Assign IRR values based on ExpansionHunter output
    entries = repeat['raw_data_tropic'].split(',')
    for entry in entries:
        f_name, irr_value = entry.split(':')
        if f_name.startswith('control_'):
            irr_values[f_name] = float(irr_value)

    # avg = np.mean(list(irr_values.values()))
    cutoff = np.percentile(list(irr_values.values()), PERCENTILE)
    
    return cutoff

merged['control_irr_cutoff'] = merged.apply(get_control_avg_irr_val, axis=1)

In [ ]:
rre = merged[['chr', 'start', 'stop', 'motif', 'cancer', 'control_irr_cutoff']].copy()

In [ ]:
rre['chr'] = 'chr' + rre['chr'].astype(str)
len(filtered_loci)

#### Add some columns we'll use later

In [ ]:
rre['locus_id'] = rre['chr'] + '_' + rre['start'].astype(str) + '_' + rre['stop'].astype(str)
rre['locus_chr'] = rre['chr']

# Expand the window by +/- 1kbp because this is how Yuen did it and we
# need to match our Locus IDs with his Cancer IDS
rre['locus_start'] = rre['start'].astype(int) - 1000
rre['locus_stop'] = rre['stop'].astype(int) + 1000

rre.head()

#### Load the 1000Genomes IRR table

In [ ]:
irr = pd.read_table('data/input/rREs-v1.0.0.hg38.padded.EHdn_overlap.tsv')

#### Split the Cancer_ID & EHdn_ID columns

In [ ]:
irr[['cancer_chr', 'cancer_start', 'cancer_stop', 'cancer_motif', 'cancer_type']] = irr['Cancer_ID'].str.split(':', n=4, expand=True)

In [ ]:
irr[['ehdn_chr', 'ehdn_start', 'ehdn_stop', 'ehdn_motif']] = irr['EHdn_ID'].str.split(':', n=3, expand=True)

#### Conver to numeric types

In [ ]:
irr[['ehdn_start', 'ehdn_stop']] = irr[['ehdn_start', 'ehdn_stop']].apply(pd.to_numeric)

In [ ]:
irr[['cancer_start', 'cancer_stop']] = irr[['cancer_start', 'cancer_stop']].apply(pd.to_numeric)

#### Liftover

In [ ]:
lo = LiftOver('hg19', 'hg38')

In [ ]:
rre[['locus_start_hg38', 'locus_stop_hg38']] = -1
        
for i, row in rre.iterrows():
    chr, start, stop = row['locus_chr'], row['locus_start'], row['locus_stop']
    possible_start = lo.convert_coordinate(chr, start)
    possible_stop = lo.convert_coordinate(chr, stop)
    if possible_start and possible_stop:
        # make sure liftover chr matches original
        assert(possible_start[0][0] == chr and possible_stop[0][0] == chr) 
        rre.loc[i, 'locus_start_hg38'] = possible_start[0][1]
        rre.loc[i, 'locus_stop_hg38'] = possible_stop[0][1]

#### Merge IRR table with table that matched Cancer_ID in Yuen's file with our locus IDs

In [ ]:
merged = irr.merge(rre, right_on=['locus_chr', 'locus_start_hg38', 'locus_stop_hg38'], left_on=['cancer_chr', 'cancer_start', 'cancer_stop'])

#### Convert IRR values to 100bp

In [ ]:
sample_cols = merged.columns[3:-21]
merged[sample_cols] = 0.5 + 1.5 * merged[sample_cols]

In [ ]:
merged.head()

#### Replace IRR values with 0 for not expanded and 1 for expanded

In [ ]:
sample_cols = merged.columns[3:-21]

for col in sample_cols:
    merged[col] = merged[col].astype(float) > merged['control_irr_cutoff']

#### Find number of expansions for each row (locus) and then percentage

We set the number of expansions to 0 if the motif does not match (because that indicates the rRE is not actually present)

In [ ]:
merged['n_expanded'] = merged[sample_cols].sum(axis=1).where(merged['Motif_match'], 0)

In [ ]:
merged['pct_expanded'] = merged['n_expanded'] / len(sample_cols)

In [ ]:
merged.head()

In [ ]:
result = merged[['locus_id', 'Cancer_ID', 'EHdn_ID', 'Motif_match', 'control_irr_cutoff', 'n_expanded', 'pct_expanded']]

#### For every locus_id, leave only one EHdn_ID (or row) and have it be the Motif_match=True if there is one

In [ ]:
result = result.sort_values('Motif_match', ascending=False).drop_duplicates('locus_id').sort_values('pct_expanded', ascending=False).reset_index(drop=True)

#### Remove loci that do not have matching motif

In [ ]:
result = result[result['Motif_match'] == True].reset_index(drop=True)

In [ ]:
result.to_csv(f'data/output/2022-10-14-rre_1000g-irr-percentile-{PERCENTILE}.tsv', index=False, sep='\t')

In [ ]:
result.head()

In [ ]:
result.sort_values('pct_expanded', ascending=False)

In [ ]:
result['pct_expanded'].describe()

In [ ]:
pdf = PdfPages(f'data/plot/2022-10-14-rre-expansion-1000g-percentile-{PERCENTILE}.pdf')
ax = sns.histplot(x='pct_expanded', data=result, binwidth=0.05, color='black')
ax.set_title(f'Threshold = {PERCENTILE}th Percentile')
ax.set_xlabel('Percentage of the population with the expansion')
ax.grid(False)
sns.despine(ax=ax, top=True, right=True)
pdf.savefig()
pdf.close()

### (4A) LRDN Benchmark Global Norm IRR count

Refer to the `ehdn-lrdn-benchmark` directory in the repository root.

### (4B) LRDN Benchmark Local Norm IRR count

Refer to the `ehdn-lrdn-benchmark` directory in the repository root.

### (5C) Correlation of rREs with Mutational Signatures

#### Prepare data

In [ ]:
sbs = pd.read_csv('data/input/mutational_signatures/PCAWG_sigProfiler_SBS_signatures_in_samples.csv')
sbs = sbs.rename({'Cancer Types': 'cancer', 'Sample Names': 'sample_id', 'Accuracy': 'accuracy'}, axis='columns')
sbs = sbs[sbs['accuracy'] > 0.95]

In [ ]:
dbs = pd.read_csv('data/input/mutational_signatures/PCAWG_sigProfiler_DBS_signatures_in_samples.csv')
dbs = dbs.rename({col: col.strip() for col in dbs.columns}, axis='columns')
dbs = dbs.rename({'Cancer Types': 'cancer', 'Sample Names': 'sample_id', 'Accuracy': 'accuracy'}, axis='columns')
dbs = dbs[dbs['accuracy'] > 0.95]

In [ ]:
ids = pd.read_csv('data/input/mutational_signatures/PCAWG_SigProfiler_ID_signatures_in_samples.csv')
ids = ids.rename({'Cancer Types': 'cancer', 'Sample Names': 'sample_id', 'Accuracy': 'accuracy'}, axis='columns')
ids = ids[ids['accuracy'] > 0.95]

In [ ]:
donors = pd.read_csv('data/input/rREs_manuscript_samples_analyzed.tsv', sep='\t')
donors = donors.rename({'donor_id/donor_count': 'donor_id', 'Specimen ID': 'sample_id', 'Number of rREs Detected': 'n_rre'}, axis='columns')
donors = donors[['sample_id', 'donor_id', 'n_rre']]
donors = donors[~donors['n_rre'].isna()]

In [ ]:
age = pd.read_excel('data/input/PCAWG7_age_information.xlsx')
age.columns = ['sample_id', 'age']

In [ ]:
from functools import reduce
dfs = [dbs, ids, sbs, donors, age]
df = reduce(lambda  left,right: pd.merge(left, right, on='sample_id', suffixes=(None, '_x')), dfs)

In [ ]:
mut_sig_cols = [col for col in df.columns if col.startswith('SBS') or col.startswith('DBS') or col.startswith('ID')]

In [ ]:
df = df[['donor_id', 'cancer', 'n_rre', 'age'] + mut_sig_cols]

#### Run predictor selection

In [ ]:
potential_cols = mut_sig_cols + ['age']

def get_rsquared(cols):
    if len(cols) > 0:
        X = sm.add_constant(df[cols])
        y = df['n_rre']
        model_w_const = sm.OLS(y, X).fit()
        
        X = df[cols]
        y = df['n_rre']
        model_no_const = sm.OLS(y, X).fit()
        
        return max(model_w_const.rsquared, model_no_const.rsquared)
    else:
        return 0

def column_selection(cols=[], i=0):
    if i == len(potential_cols):
        return get_rsquared(cols), cols
    
    include_r, include_cols =  column_selection(cols + [potential_cols[i]], i+1)
    exclude_r, exclude_cols =  column_selection(cols, i+1)
    
    if include_r > exclude_r:
        return include_r, include_cols
    else:
        return exclude_r, exclude_cols
    
column_selection()

#### Model with Age and DBS as predictors

In [ ]:
X = sm.add_constant(df[['DBS2', 'age']])
y = df['n_rre']

model = sm.OLS(y, X).fit()
model.summary()

#### Model with only DBS2 as a predictor

In [ ]:
X = sm.add_constant(df[['DBS2']])
y = df['n_rre']

model = sm.OLS(y, X).fit()
model.summary()

#### Make PDFs

In [ ]:
pdf = PdfPages('data/plot/rREs_DBS2_regression.pdf')

In [ ]:
sns.set_theme(style='white')

fig, ax = plt.subplots()

df['DBS2+age_var'] = -0.0040 * df['DBS2'] + 0.0649 * df['age'] + 0.8133

sns.regplot(x='DBS2+age_var', y='n_rre', data=df, ax=ax, color='black', scatter_kws={'color': 'grey'}, ci=None)
ax.set_ylabel('True No. of rREs')

ax.set_title('r^2=0.120')
ax.set_xlabel('Predicted No. of rRES\n(-0.0040 * DBS2 + 0.0649 * Age + 0.8133)')

sns.despine()

ax.set_ylim(0, 25)
fig.tight_layout()

pdf.savefig()

In [ ]:
sns.set_theme(style='white')

fig, ax = plt.subplots()

sns.regplot(x='DBS2', y='n_rre', data=df, ax=ax, color='black', scatter_kws={'color': 'grey'}, ci=None)
ax.set_ylabel('No. of rREs')
ax.set_xlabel('DBS')

ax.set_title('r^2=0.095')

sns.despine()

ax.set_ylim(0, 25)

pdf.savefig()

In [ ]:
pdf.close()

### (5D) Correlation of rREs with Mutational Signatures Excluding Lung-SCC rRE

In [ ]:
pdf = PdfPages('data/plot/rREs_DBS2_regression_wo_lung.pdf')

In [ ]:
df = df[df['cancer'] != 'Lung-SCC']

In [ ]:
sns.set_theme(style='white')

fig, ax = plt.subplots()

df['DBS2+age_var'] = 0.0003 * df['DBS2'] + 0.0619 * df['age'] + 0.7879

sns.regplot(x='DBS2+age_var', y='n_rre', data=df, ax=ax, color='black', scatter_kws={'color': 'grey'}, ci=None)
ax.set_ylabel('True No. of rREs')

ax.set_xlabel('Predicted No. of rRES\n(0.0003 * DBS2 + 0.0619 * Age + 0.7879)')
ax.set_title('r^2=0.025')

sns.despine()

ax.set_ylim(0, 25)
fig.tight_layout()

pdf.savefig()

In [ ]:
sns.set_theme(style='white')

fig, ax = plt.subplots()

sns.regplot(x='DBS2', y='n_rre', data=df, ax=ax, color='black', scatter_kws={'color': 'grey'}, ci=None)
ax.set_ylabel('No. of rREs')
ax.set_xlabel('DBS')

ax.set_title('r^2=0.002')

sns.despine()

ax.set_ylim(0, 25)

pdf.savefig()

### (6B) Replication timing

In [ ]:
!module load bedtools && bedtools intersect -a data/input/replication_timing.bed -b data/input/rre.bed > data/output/rre_repl_time.bed

In [ ]:
!module load bedtools && bedtools intersect -a data/input/replication_timing.bed -b data/input/simplerepeats_clean.sorted.bed > data/output/simplerepeats_repl_time.bed

In [ ]:
re = pd.read_csv("data/output/rre_repl_time.bed", sep='\t', header=None, names=['chr', 'start', 'stop', 'timing'])
simple = pd.read_csv('data/output/simplerepeats_repl_time.bed', sep='\t', header=None, names=['chr', 'start', 'stop', 'timing'])

In [ ]:
re['type'] = 'rREs'
simple['type'] = 'Simple Repeats'

In [ ]:
raw_data_arr = [simple, re]
raw_data = pd.concat(raw_data_arr)

In [ ]:
raw_data = raw_data[raw_data['timing'] != 'Undetermined']

In [ ]:
data = raw_data[['type', 'timing']].groupby(['type', 'timing']).size().reset_index(name='count')

In [ ]:
for t in list(data['type'].unique()):
    subset = data[data['type'] == t]
    total = subset['count'].sum()
    data.loc[data['type'] == t, 'pct'] = data['count'] / total

data['pct'] *= 100
data.head()

In [ ]:
early = data[data['timing'] == 'Early']
late = data[data['timing'] == 'Late']

first = early[['type', 'pct']].copy().reset_index()

second = late[['type', 'pct']].copy().reset_index()
second['pct'] += first['pct']

In [ ]:
pdf = PdfPages('data/plot/2022-10-14-distribution_repl_time.pdf') 

In [ ]:
fig, ax = plt.subplots()

palette = sns.color_palette('bright')

colors = [palette[7], palette[1], palette[2], 'white']

order = ['Simple Repeats', 'rREs']

sns.barplot(x='type', y='pct', data=second, color=colors[1], edgecolor=colors[3], order=order, ax=ax)
sns.barplot(x='type', y='pct', data=first, color=colors[2], edgecolor=colors[3], order=order, ax=ax)

late_patch = matplotlib.patches.Patch(color=colors[1], label='Late')
early_patch = matplotlib.patches.Patch(color=colors[2], label='Early')

ax.legend(handles=[late_patch, early_patch])


fig.suptitle('Distriubtion of Replication Timing')
ax.set_xlabel('')
ax.set_ylabel('Percentage (%)')

pdf.savefig(transparent=True)

In [ ]:
# Bootstrap p-value
np.random.seed(1)

N = 10000
n_sample_each_time = 35+19 # No. of repeat expansions that are either early or late

obs_early_count = data.loc[(data['type'] == 'rREs') & (data['timing'] == 'Early'), 'count'].iloc[0]
obs_late_count = data.loc[(data['type'] == 'rREs') & (data['timing'] == 'Late'), 'count'].iloc[0]

exp_early_count = data.loc[(data['type'] == 'Simple Repeats') & (data['timing'] == 'Early'), 'count'].iloc[0]
exp_late_count = data.loc[(data['type'] == 'Simple Repeats') & (data['timing'] == 'Late'), 'count'].iloc[0]

# Probability of observing "early region" for repeat expansions (observation) and simple repeats (expected)
p_observed = obs_early_count / (obs_early_count + obs_late_count)
p_expected = exp_early_count / (exp_early_count + exp_late_count)

chi_p_values = np.zeros(N)
chi_cumulative = np.zeros(N)

t_p_values = np.zeros(N)
t_cumulative = np.zeros(N)

for i in range(N):
    # Take 'n_sample_each_time' samples and record how many of them are "early regions" and how many are "late"
    n_early_obs = np.random.binomial(n_sample_each_time, p_observed)
    n_early_exp = np.random.binomial(n_sample_each_time, p_expected)
    n_late_obs = n_sample_each_time - n_early_obs
    n_late_exp = n_sample_each_time - n_early_exp
    
    # Calculate a chisqure p_value of observering these frequencies
    _, p_value = stats.chisquare([n_early_obs, n_late_obs], f_exp=[n_early_exp, n_late_exp])
    chi_p_values[i] = p_value
    chi_cumulative[i] = np.mean(chi_p_values[:i+1])
    
    # Calculate a ttest p-value
    obs_distribtion = np.zeros(n_sample_each_time)
    exp_distribtion = np.zeros(n_sample_each_time)
    
    obs_distribtion[:n_early_obs] = 1
    exp_distribtion[:n_early_exp] = 1
    
    _, p_value = stats.ttest_ind(obs_distribtion, exp_distribtion, equal_var=False)
    t_p_values[i] = p_value
    t_cumulative[i] = np.mean(t_p_values[:i+1])
    
    
t_cumulative[N-1], chi_cumulative[N-1]

In [ ]:
sns.scatterplot(x=np.arange(N), y=t_cumulative)

In [ ]:
sns.scatterplot(x=np.arange(N), y=chi_cumulative)

In [ ]:
fig = plt.figure()
fig.clf()

text = 'Bootstrapped chi-square test p-value: {:e}\n'.format(chi_cumulative[N-1])
text += 'Bootstrapped student t-test p-value: {:e}\n'.format(t_cumulative[N-1])


fig.text(0.5,0.5, text, transform=fig.transFigure, size=12, ha='center')
pdf.savefig()
plt.close()

In [ ]:
pdf.close()

### (7A) Overlap of rREs with published datasets

In [ ]:
park = pd.read_table('data/input/published-datasets/Park-Top-1000-recurrently-altered-MSs-Suppleementary-Data-14-41467_2017_BFncomms15180_MOESM272_ESM (1).xls')
park = park[['CHR', 'START', 'END']]
park.columns = ['chr', 'start', 'stop']
park = park.drop_duplicates(ignore_index=True)

# park.head()

In [ ]:
mischel = pd.read_csv('data/input/published-datasets/Mischel-Data-Supplementary-Table-1-41586_2019_1763_MOESM3_ESM.csv')
mischel = mischel[['Chr', 'Start', 'End']]
mischel.columns = ['chr', 'start', 'stop']

mischel = mischel.dropna()
mischel['chr'] = 'chr' + mischel['chr'].astype('int').astype('str')
mischel['start'] = mischel['start'].astype('int')
mischel['stop'] = mischel['stop'].astype('int')

mischel = mischel.drop_duplicates(ignore_index=True)

# mischel.head()

In [ ]:
shendure = pd.read_excel('data/input/published-datasets/Hause-Shendure-Supplementary-Table-10-MSI-H-cancer-specific-instability-loci-41591_2016_BFnm4191_MOESM30_ESM.xlsx')
shendure[['chr', 'pos']] = shendure['locus'].str.split(':', n=2, expand=True)
shendure[['start', 'stop']] = shendure['pos'].str.split('-', n=2, expand=True)

shendure = shendure[['chr', 'start', 'stop']]

shendure['chr'] = 'chr' + shendure['chr'].astype('str')
shendure['start'] = shendure['start'].astype('int')
shendure['stop'] = shendure['stop'].astype('int')

shendure = shendure.drop_duplicates(ignore_index=True)

# shendure.head()

In [ ]:
gymrek = pd.read_excel('data/input/published-datasets/Gymrek-Supplementary-Data-1-41588_2019_521_MOESM4_ESM.xlsx', sheet_name=1)
gymrek = gymrek[['chrom', 'str.start', 'str.end']]
gymrek.columns = ['chr', 'start', 'stop']

gymrek = gymrek.drop_duplicates(ignore_index=True)

# gymrek.head()

In [ ]:
dutta = None

import glob
for path in glob.glob('data/input/published-datasets/GSM*'):
    df = pd.read_table(path, skiprows=1, header=None)
    if dutta is not None:
        dutta = pd.concat([dutta, df], ignore_index=True)
    else:
        dutta = df

dutta.columns = ['chr', 'start', 'stop', 'abundance']
dutta = dutta[['chr', 'start', 'stop']]
dutta = dutta.drop_duplicates(ignore_index=True)

# dutta.head()

#### Export bed files

In [ ]:
dfs = [park, mischel, shendure, gymrek, dutta]
labels = ['Park', 'Mischel', 'Shendure', 'Gymrek', 'Dutta']

In [ ]:
for i in range(len(dfs)):
    dfs[i].to_csv(f'data/input/published-datasets/bed/{labels[i]}.bed', header=None, sep='\t', index=False)

In [ ]:
for f in labels:
    !sort -k1,1 -k2,2n data/input/published-datasets/bed/{f}.bed > data/input/published-datasets/bed/{f}.sorted.bed 

#### Run bedtools

In [ ]:
for f in labels:
    !bedtools intersect -sorted -u -b data/input/rre.bed -a data/input/published-datasets/bed/{f}.sorted.bed > data/output/{f}-rre-overlap.bed 

#### Extract percent overlap

In [ ]:
from pandas.errors import EmptyDataError

pct = [-1] * len(labels)
for i in range(len(labels)):
    try:
        df = pd.read_table(f'data/output/{labels[i]}-rre-overlap.bed', header=None)
        pct[i] = len(df) / len(RRE)
    except EmptyDataError as e:
        pct[i] = 0

pct

#### Plot

In [ ]:
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

In [ ]:
data = pd.DataFrame({'catalog': labels, 'pct': pct})

In [ ]:
data

In [ ]:
pdf = PdfPages(f'data/plot/2022-10-14-published-catalog-overlap.pdf')

ax = sns.barplot(x='catalog', y='pct', data=data, color='black')
ax.set_xlabel('')
ax.set_ylabel('Percent Overlap')
ax.grid(False)
ax.set_ylim(0, 1)
sns.despine(ax=ax, top=True, right=True)
change_width(ax, .35)
pdf.savefig()

pdf.close()

### (7C) Distance to Nearest cCRE (separated by cCRE type)

In [ ]:
rre_distance = pd.read_table("data/output/closest_rre_ccre.bed", header=None)
simple_distance = pd.read_table("data/output/closest_simplerepeats_ccre.bed", header=None)

# Extract distance column from bedtools output
type_col = 8
distance_col = 9
rre_distance = rre_distance[[distance_col, type_col]].rename({distance_col: "distance", type_col: "ccre"}, axis="columns")
simple_distance = simple_distance[[distance_col, type_col]].rename({distance_col: "distance", type_col: "ccre"}, axis="columns")

rre_distance["type"] = "rREs"
simple_distance["type"] = "Simple Repeats"

# Convert distance to kbp
rre_distance["distance"] = rre_distance["distance"] / 10**3
simple_distance["distance"] = simple_distance["distance"] / 10**3

simple_distance = simple_distance[simple_distance["distance"] < 25000]

# Merge the two dataframes
data = pd.concat([rre_distance, simple_distance], ignore_index=True)

In [ ]:
pdf = PdfPages("data/plot/2022-10-17-distance-to-nearest-ccre-by-type.pdf")

fig, ax = plt.subplots()
sns.boxplot(x="type", y="distance", order=["Simple Repeats", "rREs"], hue="ccre", showfliers=False, width=0.4, data=data, ax=ax)
ax.set_title(f"Distance to Nearest cCRE")
ax.set_xlabel("")
ax.set_ylabel("Distance (kbp)")

sns.despine()
# plt.setp(ax.artists, edgecolor = 'k', facecolor='w')
# plt.setp(ax.lines, color='k')
ax.grid(False)
plt.tight_layout()

pdf.savefig(transparent=True)
pdf.close()

### (8C) UGT2B7 Survival

In [ ]:
paired_path = "data/input/ugt2b7_survival/Kidney-RCC_clinic_paired_AAAG_4_69929297_69930148.txt"
donor_path = "data/input/ugt2b7_survival/Kidney-RCC_donor.tsv"

In [ ]:
pdf = PdfPages('data/plot/ugt2b7_survival.pdf')

In [ ]:
paired = pd.read_csv(paired_path, sep='\t')
donor = pd.read_csv(donor_path, sep='\t')
donor.rename({'icgc_donor_id': 'case_id'}, axis='columns', inplace=True)
paired = paired.merge(donor, how='left', on='case_id')

meta = {
    "chr": "chr4",
    "start": "69929297",
    "stop": "69930148"
}

paired = paired[~paired['donor_survival_time'].isna() & ~paired['donor_vital_status'].isna()]

exp = paired[paired['exp'] == 'Yes']
exp_time = paired['donor_survival_time'].astype(int).values
exp_status = (paired['donor_vital_status'] == 'alive').astype(int).values

no_exp = paired[paired['exp'] == 'No']
no_exp_time = no_exp['donor_survival_time'].astype(int).values
no_exp_status = (no_exp['donor_vital_status'] == 'alive').astype(int).values

# Fit model for expansion
kmf_exp = KaplanMeierFitter() 
kmf_exp.fit(exp_time, exp_status, label='Expansion (n = %d)' % exp.shape[0])

# Fit model for no expansion
kmf_no_exp = KaplanMeierFitter() 
kmf_no_exp.fit(no_exp_time, no_exp_status, label='No Expansion (n = %d)' % no_exp.shape[0])

stats = logrank_test(exp_time, no_exp_time, exp_status, no_exp_status)

fig, ax = plt.subplots(figsize=(8, 6))

ax = kmf_exp.plot(ci_show=False, ax=ax)
ax = kmf_no_exp.plot(ci_show=False, ax=ax)

stats = logrank_test(exp_time, no_exp_time, exp_status, no_exp_status)

ax.set_xlabel('')
add_at_risk_counts(kmf_exp, kmf_no_exp, ax=ax)
title = [
    'Kaplan Meier Curve (in days)',
    'Repeat coordinates: {}: {}-{}'.format(meta['chr'], meta['start'], meta['stop']),
    'p={:.4e}'.format(stats.p_value)
]
ax.set_title('\n'.join(title))    
plt.tight_layout(pad=5)

pdf.savefig(fig)
plt.close(fig)

In [ ]:
pdf.close()